In [1]:
# Imports

import json
import geojson
import geopandas
import csv
import copy
import ast
import os
from collections import OrderedDict

from geojson import Feature, Point, FeatureCollection


In [2]:
fips_codes = {
 '01': {'abbr': 'AL', 'full_name': 'alabama'},
 '02': {'abbr': 'AK', 'full_name': 'alaska'},
 '04': {'abbr': 'AZ', 'full_name': 'arizona'},
 '05': {'abbr': 'AR', 'full_name': 'arkansas'},
 '06': {'abbr': 'CA', 'full_name': 'california'},
 '08': {'abbr': 'CO', 'full_name': 'colorado'},
 '09': {'abbr': 'CT', 'full_name': 'connecticut'},
 '10': {'abbr': 'DE', 'full_name': 'delaware'},
 '11': {'abbr': 'DC', 'full_name': 'district_of_columbia'},
 '12': {'abbr': 'FL', 'full_name': 'florida'},
 '13': {'abbr': 'GA', 'full_name': 'georgia'},
 '15': {'abbr': 'HI', 'full_name': 'hawaii'},
 '16': {'abbr': 'ID', 'full_name': 'idaho'},
 '17': {'abbr': 'IL', 'full_name': 'illinois'},
 '18': {'abbr': 'IN', 'full_name': 'indiana'},
 '19': {'abbr': 'IA', 'full_name': 'iowa'},
 '20': {'abbr': 'KS', 'full_name': 'kansas'},
 '21': {'abbr': 'KY', 'full_name': 'kentucky'},
 '22': {'abbr': 'LA', 'full_name': 'louisiana'},
 '23': {'abbr': 'ME', 'full_name': 'maine'},
 '24': {'abbr': 'MD', 'full_name': 'maryland'},
 '25': {'abbr': 'MA', 'full_name': 'massachusetts'},
 '26': {'abbr': 'MI', 'full_name': 'michigan'},
 '27': {'abbr': 'MN', 'full_name': 'minnesota'},
 '28': {'abbr': 'MS', 'full_name': 'mississippi'},
 '29': {'abbr': 'MO', 'full_name': 'missouri'},
 '30': {'abbr': 'MT', 'full_name': 'montana'},
 '31': {'abbr': 'NE', 'full_name': 'nebraska'},
 '32': {'abbr': 'NV', 'full_name': 'nevada'},
 '33': {'abbr': 'NH', 'full_name': 'new_hampshire'},
 '34': {'abbr': 'NJ', 'full_name': 'new_jersey'},
 '35': {'abbr': 'NM', 'full_name': 'new_mexico'},
 '36': {'abbr': 'NY', 'full_name': 'new_york'},
 '37': {'abbr': 'NC', 'full_name': 'north_carolina'},
 '38': {'abbr': 'ND', 'full_name': 'north_dakota'},
 '39': {'abbr': 'OH', 'full_name': 'ohio'},
 '40': {'abbr': 'OK', 'full_name': 'oklahoma'},
 '41': {'abbr': 'OR', 'full_name': 'oregon'},
 '42': {'abbr': 'PA', 'full_name': 'pennsylvania'},
 '44': {'abbr': 'RI', 'full_name': 'rhode_island'},
 '45': {'abbr': 'SC', 'full_name': 'south_carolina'},
 '46': {'abbr': 'SD', 'full_name': 'south_dakota'},
 '47': {'abbr': 'TN', 'full_name': 'tennessee'},
 '48': {'abbr': 'TX', 'full_name': 'texas'},
 '49': {'abbr': 'UT', 'full_name': 'utah'},
 '50': {'abbr': 'VT', 'full_name': 'vermont'},
 '51': {'abbr': 'VA', 'full_name': 'virginia'},
 '53': {'abbr': 'WA', 'full_name': 'washington'},
 '54': {'abbr': 'WV', 'full_name': 'west_virginia'},
 '55': {'abbr': 'WI', 'full_name': 'wisconsin'},
 '56': {'abbr': 'WY', 'full_name': 'wyoming'}}

In [3]:
# Input Files
# 2 => "02"
def formatted_state_fips_code(fips_number):
    return f'{fips_number:02d}'

#  block_group_geojson_filename(state_fips_code=46, year=2010)
# tl_2010_46_bg10.json
def block_group_geojson_filename(state_fips_code=1, census_year=2010):
    year_string = str(census_year)
    fips_code_string = formatted_state_fips_code(state_fips_code)
    return f'tl_{year_string}_{fips_code_string}_bg10.json'

# wa_washington_zip_codes_geo.min.json
def zip_code_geojson_filename(state_fips_code=1):
    formatted_fips_code = formatted_state_fips_code(state_fips_code)
    state_abbr = fips_codes[formatted_fips_code]["abbr"].lower()
    state_full_name = fips_codes[formatted_fips_code]["full_name"]
    return f'{state_abbr}_{state_full_name}_zip_codes_geo.min.json'


def zip_geojson_files_for_market(market_data):
    fips_codes = json.loads(market_data['StateFips'])
    files = []
    for fips_code in fips_codes:
        files.append(zip_code_geojson_filename(fips_code))
    return files

def bg_geojson_files_for_market(market_data, census_year):
    fips_codes = json.loads(market_data['StateFips'])
    files = []
    for fips_code in fips_codes:
        files.append(block_group_geojson_filename(state_fips_code=int(fips_code), census_year=census_year))
    return files

def read_geo_gson_file(file_path):
    with open(file_path) as f:
        gj = geojson.load(f)
        features = gj['features']
    print(f'Loaded {len(features)} features from {file_path}')
    return  features

def read_geojson_files(file_paths, base_path = "./"):
    features = []
    for file_path in file_paths:
        features += read_geo_gson_file(os.path.join(base_path, file_path))
    return features
    

def read_market_zip_code_usage(market_file_prefix, base_path="./source_data" ):
    data_type = "zip"
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
    data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    zip_codes = []
    for data_row in data_rows:
        zip_codes.append(data_row['Zip'].zfill(5))
    print(f'Loaded {len(zip_codes)} zip_codes from {csv_file_path}')
    return zip_codes
        
def read_market_block_group_usage(market_file_prefix, base_path="./source_data" ):
    data_type = "bg"
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
    data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    block_groups = []
    for data_row in data_rows:
        block_groups.append(data_row['BG'].zfill(12))
    print(f'Loaded {len(block_groups)} block groups from {csv_file_path}')
    return zip_codes
        
# market_file_prefix = 'DallasFortWorth'
# market_prefix= 'dfw'
# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes

def file_type_for_data_type(data_type = "zip"):
    if data_type == "zip": 
        return "Zip"
    if data_type == "bg":
        return "BG"

# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
def mapping_data_file_path(market_file_prefix, base_path="./source_data", data_type = "zip"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}__Mapping_{file_type}_Data.csv'

# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes
def map_settings_data_file_path(market_file_prefix, base_path="./source_data", data_type = "zip"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}__Mapping_Settings_Transposed File_{file_type}.csv'

def published_map_data_path(market_prefix,  data_type = "zip", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_{data_type}_data.json'

def published_map_settings_path(market_prefix,  data_type = "zip", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_{data_type}_settings_data.json'

def published_map_labels_path(market_prefix,  data_type = "zip", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_{data_type}_labels.json'

def read_mapping_data(market_file_prefix, base_path='./source_data', data_type='zip'):
    data_rows = []
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')
    return data_rows

def create_zip_features(features, zip_data_rows):
    new_features_to_keep = []
    for data_row in zip_data_rows:
        zip_code = data_row['Zip'].zfill(5)
        # find the associated feature
        for feature in features:
            if feature['properties']["ZCTA5CE10"] == zip_code:
                new_feature = copy.deepcopy(feature)
                for key in data_row.keys():
                    if key != 'Zip':
                        if data_row[key]:
                            if data_row[key] != 'inf':
                                new_feature['properties'][key] =  ast.literal_eval(data_row[key].replace(',',''))
                new_features_to_keep.append(new_feature)
                break
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_bg_features(features, bg_data_rows):
    new_features_to_keep = []
    for data_row in bg_data_rows:
        block_group = str(data_row['BG']).zfill(12)
        # find the associated feature
        found = False
        for feature in features:
            if feature['properties']["GEOID10"] == block_group:
                found = True
                new_feature = copy.deepcopy(feature)
                for key in data_row.keys():
                    if key != 'BG':
                        if data_row[key] and data_row[key] != 'inf':
                            new_feature['properties'][key] =  ast.literal_eval(data_row[key].replace(',',''))
                new_features_to_keep.append(new_feature)
                break
    #         if found == False:
    #             print(f'failed: {block_group}')
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_geojson(features):
    print(f'Created GeoJson with  {len(features)} features')
    return FeatureCollection(features)

def write_geojson(geojson_data, market_prefix, data_type='zip',published_data_base_path = "./public"):
    geojson_string = geojson.dumps(geojson_data, sort_keys=True)
    f = open(f'{published_data_base_path}/{market_prefix}_{data_type}_data.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geojson data to  {published_data_base_path}/{market_prefix}_{data_type}_data.json')
    
def read_map_settings(market_file_prefix, base_path="./source_data", data_type = "zip"):
    data_rows = []
    csv_file_path = map_settings_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')   
    return data_rows

def write_settings_data(settings_data, market_file_prefix, base_path="./public", data_type = "zip"):
    processed_settings_data = OrderedDict()
    for row in settings_data:
        processed_settings_data[row['Bain_Short_Name']] = row
    json_string = json.dumps(processed_settings_data, indent = 2, sort_keys=False)
    settings_data_file_path = published_map_settings_path(market_file_prefix,  data_type = data_type, published_data_base_path=base_path)
    f = open(settings_data_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote settings data rows to {settings_data_file_path}') 
    
def create_zip_labels(market_file_prefix, base_path="./source_data" , data_type = "zip" , published_data_base_path="./public"):
    data_rows = []
    #     csv_file_path = './DFW_Mapping_data.csv'
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = "zip")
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)

    features = []
    for data_row in data_rows:
        feature =  { 
            "type": "Feature", 
            "properties": {
                "id": data_row['Zip']
            },
            "geometry": { 
                "type": "Point", 
                "coordinates": [ float(data_row['intptlong']), float(data_row['intptlat']) ]
            }
        }

        features.append(feature)
    geo_json = {
        "type": "FeatureCollection",
        "features": features
    }
    json_string = json.dumps(geo_json, indent = 2, sort_keys=True)
    output_file_path = published_map_labels_path(market_prefix,  data_type , published_data_base_path="./public")
    f = open(output_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson labels data to  {output_file_path}')

def create_bg_labels(market_file_prefix, base_path="./source_data" , data_type = "bg" , published_data_base_path="./public"):
    data_rows = []
    #     csv_file_path = './DFW_Mapping_data.csv'
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = "bg")
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)

    features = []
    for data_row in data_rows:
        feature =  { 
            "type": "Feature", 
            "properties": {
                "id": data_row['BG']
            },
            "geometry": { 
                "type": "Point", 
                "coordinates": [ float(data_row['LONG']), float(data_row['LAT']) ]
            }
        }

        features.append(feature)
    geo_json = {
        "type": "FeatureCollection",
        "features": features
    }
    json_string = json.dumps(geo_json, indent = 2, sort_keys=True)
    output_file_path = published_map_labels_path(market_prefix,  data_type , published_data_base_path="./public")
    f = open(output_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson labels data to  {output_file_path}')

def create_geojson_settings(csv_file_path, published_data_base_path = "./public"):

    geospatial_settings = OrderedDict()
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            geospatial_settings[row['MarketName']] = row
    geojson_string = json.dumps(geospatial_settings, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/geospatial_settings.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geospatials settings data to  {published_data_base_path}/geospatial_settings.json')
    

In [4]:
# Create all of the data
source_data_base_path = "./source_data"
source_data_base_path = "/Users/alanmccann/Dropbox/bain/28th_run"
published_data_base_path = "./public"
geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
census_year = 2010
csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
market_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        market_rows.append(row)
market_count = 0
completed_markets = []
for market_data in market_rows:
    print("")
    print(f'Market Data: {market_count + 1}')
    print(market_data)
    market_file_prefix = market_data['MarketName']
    market_prefix = market_file_prefix
    # Zip Processing     
    data_type = 'zip'
    print(f'market: {market_prefix}')
    geojson_files = zip_geojson_files_for_market(market_data)
    zip_codes = read_market_zip_code_usage(market_file_prefix, base_path=source_data_base_path )
    zip_data_rows = read_mapping_data(market_file_prefix, source_data_base_path, data_type)
    features = read_geojson_files(geojson_files, base_path=geojson_files_base_path)
    features_to_use = create_zip_features(features, zip_data_rows)
    new_geojson = create_geojson(features_to_use)
    write_geojson(new_geojson, market_prefix, data_type, published_data_base_path=published_data_base_path)
    map_settings_data = read_map_settings(market_file_prefix, base_path=source_data_base_path, data_type = "zip")
    write_settings_data(map_settings_data, market_prefix, base_path=published_data_base_path, data_type = "zip")
    create_zip_labels(market_file_prefix, base_path=source_data_base_path , data_type = "zip" , published_data_base_path=published_data_base_path)
    # Block Group Processing
    data_type = 'bg'
    geojson_files = bg_geojson_files_for_market(market_data,census_year )
    block_groups = read_market_block_group_usage(market_file_prefix, base_path=source_data_base_path )
    block_group_rows = read_mapping_data(market_file_prefix, source_data_base_path, data_type)
    features = read_geojson_files(geojson_files, base_path=geojson_files_base_path)
    features_to_use = create_bg_features(features, block_group_rows)
    new_geojson = create_geojson(features_to_use)
    write_geojson(new_geojson, market_prefix, data_type, published_data_base_path=published_data_base_path)
    map_settings_data = read_map_settings(market_file_prefix, base_path=source_data_base_path, data_type = "bg")
    write_settings_data(map_settings_data, market_prefix, base_path=published_data_base_path, data_type = "bg")
    create_bg_labels(market_file_prefix, base_path=source_data_base_path , data_type = "bg" , published_data_base_path=published_data_base_path)
    market_count += 1
# Finished states - create overall settings file    
create_geojson_settings(csv_file_path, published_data_base_path = "./public")
print(f'*** COMPLETED {market_count} markets')
print(completed_markets)



Market Data: 1
{'MarketName': 'Albany', 'Market_Display_Name': 'Albany', 'StateFips': '[9, 25, 36, 50]', 'CenterLat': '42.675748', 'CenterLong': '-73.74473499999999', 'GeoTypeGroup': '1 - Designated Metro Areas', 'DefaultZoomLevel': '8'}
market: Albany
Loaded 201 zip_codes from /Users/alanmccann/Dropbox/bain/28th_run/Albany__Mapping_Zip_Data.csv
Loaded 201 data rows from /Users/alanmccann/Dropbox/bain/28th_run/Albany__Mapping_Zip_Data.csv
Loaded 282 features from /Users/alanmccann/Dropbox/bain/map_source_data/ct_connecticut_zip_codes_geo.min.json
Loaded 538 features from /Users/alanmccann/Dropbox/bain/map_source_data/ma_massachusetts_zip_codes_geo.min.json
Loaded 1794 features from /Users/alanmccann/Dropbox/bain/map_source_data/ny_new_york_zip_codes_geo.min.json
Loaded 255 features from /Users/alanmccann/Dropbox/bain/map_source_data/vt_vermont_zip_codes_geo.min.json
Processed 201 features to use
Created GeoJson with  201 features
Wrote geojson data to  ./public/Albany_zip_data.json
Lo

Loaded 574 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_10_bg10.json
Loaded 450 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_11_bg10.json
Loaded 3926 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_24_bg10.json
Loaded 9740 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_42_bg10.json
Processed 3656 features to use
Created GeoJson with  3656 features
Wrote geojson data to  ./public/Baltimore_bg_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/28th_run/Baltimore__Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to ./public/Baltimore_bg_settings_data.json
Wrote geojson labels data to  ./public/Baltimore_bg_labels.json

Market Data: 6
{'MarketName': 'Baton Rouge', 'Market_Display_Name': 'Baton Rouge', 'StateFips': '[22, 28]', 'CenterLat': '30.440759000000003', 'CenterLong': '-91.117696', 'GeoTypeGroup': '1 - Designated Metro Areas', 'DefaultZoomLevel': '8'}
market: Bato

Loaded 248 features from /Users/alanmccann/Dropbox/bain/map_source_data/nh_new_hampshire_zip_codes_geo.min.json
Loaded 1794 features from /Users/alanmccann/Dropbox/bain/map_source_data/ny_new_york_zip_codes_geo.min.json
Loaded 255 features from /Users/alanmccann/Dropbox/bain/map_source_data/vt_vermont_zip_codes_geo.min.json
Processed 107 features to use
Created GeoJson with  107 features
Wrote geojson data to  ./public/Burlington_zip_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/28th_run/Burlington__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/Burlington_zip_settings_data.json
Wrote geojson labels data to  ./public/Burlington_zip_labels.json
Loaded 324 block groups from /Users/alanmccann/Dropbox/bain/28th_run/Burlington__Mapping_BG_Data.csv
Loaded 324 data rows from /Users/alanmccann/Dropbox/bain/28th_run/Burlington__Mapping_BG_Data.csv
Loaded 922 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_33_bg10.json
Loade

Processed 1415 features to use
Created GeoJson with  1415 features
Wrote geojson data to  ./public/Charlotte_bg_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/28th_run/Charlotte__Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to ./public/Charlotte_bg_settings_data.json
Wrote geojson labels data to  ./public/Charlotte_bg_labels.json

Market Data: 15
{'MarketName': 'Chattanooga', 'Market_Display_Name': 'Chattanooga', 'StateFips': '[1, 13, 37, 47]', 'CenterLat': '35.07271', 'CenterLong': '-85.229766', 'GeoTypeGroup': '1 - Designated Metro Areas', 'DefaultZoomLevel': '8'}
market: Chattanooga
Loaded 90 zip_codes from /Users/alanmccann/Dropbox/bain/28th_run/Chattanooga__Mapping_Zip_Data.csv
Loaded 90 data rows from /Users/alanmccann/Dropbox/bain/28th_run/Chattanooga__Mapping_Zip_Data.csv
Loaded 645 features from /Users/alanmccann/Dropbox/bain/map_source_data/al_alabama_zip_codes_geo.min.json
Loaded 735 features from /Users/alanmccann/Dropbox/bain/map_sour

Loaded 526 features from /Users/alanmccann/Dropbox/bain/map_source_data/co_colorado_zip_codes_geo.min.json
Processed 58 features to use
Created GeoJson with  58 features
Wrote geojson data to  ./public/ColoradoSprings_zip_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/28th_run/ColoradoSprings__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/ColoradoSprings_zip_settings_data.json
Wrote geojson labels data to  ./public/ColoradoSprings_zip_labels.json
Loaded 471 block groups from /Users/alanmccann/Dropbox/bain/28th_run/ColoradoSprings__Mapping_BG_Data.csv
Loaded 471 data rows from /Users/alanmccann/Dropbox/bain/28th_run/ColoradoSprings__Mapping_BG_Data.csv
Loaded 3532 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_08_bg10.json
Processed 471 features to use
Created GeoJson with  471 features
Wrote geojson data to  ./public/ColoradoSprings_bg_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/28th_run/Colo

...

Loaded 4125 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_47_bg10.json
Processed 973 features to use
Created GeoJson with  973 features
Wrote geojson data to  ./public/Nashville_bg_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/28th_run/Nashville__Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to ./public/Nashville_bg_settings_data.json
Wrote geojson labels data to  ./public/Nashville_bg_labels.json

Market Data: 59
{'MarketName': 'NewOrleans', 'Market_Display_Name': 'New Orleans', 'StateFips': '[22]', 'CenterLat': '29.926973999999998', 'CenterLong': '-90.11586899999999', 'GeoTypeGroup': '1 - Designated Metro Areas', 'DefaultZoomLevel': '8'}
market: NewOrleans
Loaded 83 zip_codes from /Users/alanmccann/Dropbox/bain/28th_run/NewOrleans__Mapping_Zip_Data.csv
Loaded 83 data rows from /Users/alanmccann/Dropbox/bain/28th_run/NewOrleans__Mapping_Zip_Data.csv
Loaded 516 features from /Users/alanmccann/Dropbox/bain/map_source_data/la_

Loaded 935 features from /Users/alanmccann/Dropbox/bain/map_source_data/ia_iowa_zip_codes_geo.min.json
Loaded 586 features from /Users/alanmccann/Dropbox/bain/map_source_data/ne_nebraska_zip_codes_geo.min.json
Processed 125 features to use
Created GeoJson with  125 features
Wrote geojson data to  ./public/Omaha_zip_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/28th_run/Omaha__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/Omaha_zip_settings_data.json
Wrote geojson labels data to  ./public/Omaha_zip_labels.json
Loaded 844 block groups from /Users/alanmccann/Dropbox/bain/28th_run/Omaha__Mapping_BG_Data.csv
Loaded 844 data rows from /Users/alanmccann/Dropbox/bain/28th_run/Omaha__Mapping_BG_Data.csv
Loaded 2630 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_19_bg10.json
Loaded 1633 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_31_bg10.json
Processed 844 features to use
Created GeoJson with  844 

Loaded 6320 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_34_bg10.json
Loaded 9740 features from /Users/alanmccann/Dropbox/bain/map_source_data/tl_2010_42_bg10.json
Processed 4612 features to use
Created GeoJson with  4612 features
Wrote geojson data to  ./public/Philadelphia_bg_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/28th_run/Philadelphia__Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to ./public/Philadelphia_bg_settings_data.json
Wrote geojson labels data to  ./public/Philadelphia_bg_labels.json

Market Data: 68
{'MarketName': 'Phoenix', 'Market_Display_Name': 'Phoenix', 'StateFips': '[4]', 'CenterLat': '33.592517', 'CenterLong': '-112.088004', 'GeoTypeGroup': '1 - Designated Metro Areas', 'DefaultZoomLevel': '8'}
market: Phoenix
Loaded 132 zip_codes from /Users/alanmccann/Dropbox/bain/28th_run/Phoenix__Mapping_Zip_Data.csv
Loaded 132 data rows from /Users/alanmccann/Dropbox/bain/28th_run/Phoenix__Mapping_Zip_Data.cs

Loaded 282 features from /Users/alanmccann/Dropbox/bain/map_source_data/ct_connecticut_zip_codes_geo.min.json
Loaded 538 features from /Users/alanmccann/Dropbox/bain/map_source_data/ma_massachusetts_zip_codes_geo.min.json
Loaded 77 features from /Users/alanmccann/Dropbox/bain/map_source_data/ri_rhode_island_zip_codes_geo.min.json
Processed 295 features to use
Created GeoJson with  295 features
Wrote geojson data to  ./public/ProvidenceNew Bedford_zip_data.json
Loaded 38 data rows from /Users/alanmccann/Dropbox/bain/28th_run/ProvidenceNew Bedford__Mapping_Settings_Transposed File_Zip.csv
Wrote settings data rows to ./public/ProvidenceNew Bedford_zip_settings_data.json
Wrote geojson labels data to  ./public/ProvidenceNew Bedford_zip_labels.json
Loaded 3165 block groups from /Users/alanmccann/Dropbox/bain/28th_run/ProvidenceNew Bedford__Mapping_BG_Data.csv
Loaded 3165 data rows from /Users/alanmccann/Dropbox/bain/28th_run/ProvidenceNew Bedford__Mapping_BG_Data.csv
Loaded 2585 features fro

KeyboardInterrupt: 

In [6]:
# Chart data
csv_file_path = '/Users/alanmccann/Dropbox/bain/Chart_Data_File.csv'
chart_data_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        chart_data_rows.append(row)
chart_data_rows

FileNotFoundError: [Errno 2] No such file or directory: '/Users/alanmccann/Dropbox/bain/Chart_Data_File.csv'

In [7]:
data_json = {}
for row in chart_data_rows:
    region = row['Region']
    segment = row['Segment']
    year = row['Year']
    urban = row['Urban']
    suburban = row['SubUrban']
    exurban = row['ExUrban']
    if region not in data_json.keys():
        data_json[region] = {}
    if year not in data_json[region].keys():
        data_json[region][year] = {}
    if 'Urban' not in data_json[region][year].keys():
        data_json[region][year]['Urban'] = {}
        data_json[region][year]['SubUrban'] = {}
        data_json[region][year]['ExUrban'] = {}
    data_json[region][year]['Urban'][segment] = urban
    data_json[region][year]['SubUrban'][segment] = suburban
    data_json[region][year]['ExUrban'][segment] = exurban
data_json
    
        
        

{}

In [10]:


def create_chart_json(csv_file_path, published_data_base_path = "./public"):

    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        segment = row['Segment']
        year = row['Year']
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        display_region = row['DisplayRegion']
        if region not in data_json.keys():
            data_json[region] = {'displayRegion': display_region, 'data': {}}
        if year not in data_json[region]['data'].keys():
            data_json[region]['data'][year] = {}
        if 'Urban' not in data_json[region]['data'][year].keys():
            data_json[region]['data'][year]['Urban'] = {}
            data_json[region]['data'][year]['SubUrban'] = {}
            data_json[region]['data'][year]['ExUrban'] = {}
        data_json[region]['data'][year]['Urban'][segment] = urban
        data_json[region]['data'][year]['SubUrban'][segment] = suburban
        data_json[region]['data'][year]['ExUrban'][segment] = exurban
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_summary_charts.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_summary_charts.json')
source_data_base_path = "/Users/alanmccann/Dropbox/bain/28th_run"
csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
create_chart_json(csv_file_path)

Wrote market summary charts data to  ./public/market_summary_charts.json
